## Pandas analysis

This exercise consists in analyzing a dataset containg timing information from a series of Time-to-Digital-Converters (TDC) implemented in a couple of FPGAs. Each measurement (i.e. each row of the input file) consists of a flag that specifies the type of message ('HEAD', which in this case is always 1), two addresses of the TDC providing the signal ('FPGA' and 'TDC_CHANNEL'), and the timing information ('ORBIT_CNT', 'BX_COUNTER', and 'TDC_MEAS'). Each TDC count corresponds to 25/30 ns, whereas a unit of BX_COUNTER corresponds to 25 ns, and the ORBIT_CNT is increased every 'x' BX_COUNTER. This allows to store the time in a similar way to hours, minutes and seconds.

In [59]:
# If you didn't download it yet, please get the relevant file now!
!wget https://www.dropbox.com/s/xvjzaxzz3ysphme/data_000637.txt -P ~/data/


'wget' is not recognized as an internal or external command,
operable program or batch file.


1\. Create a Pandas DataFrame reading N rows of the 'data_000637.txt' dataset. Choose N to be smaller than or equal to the maximum number of rows and larger that 10k.

2\. Find out the number of BX in a ORBIT (the value 'x').

3\. Find out how much the data taking lasted. You can either make an estimate based on the fraction of the measurements (rows) you read, or perform this check precisely by reading out the whole dataset.

4\. Create a new column with the absolute time in ns (as a combination of the other three columns with timing information).

5\. Replace the values (all 1) of the HEAD column randomly with 0 or 1.

6\. Create a new DataFrame that contains only the rows with HEAD=1.

7\. Make two occupancy plots (one for each FPGA), i.e. plot the number of counts per TDC channel

8\. Use the groupby method to find out the noisy channels, i.e. the TDC channels with most counts (say the top 3)

9\. Count the number of unique orbits. Count the number of unique orbits with at least one measurement from TDC_CHANNEL=139

# explanation notes
every record is a series of variable, 
gives address of the measurement and the time
address is split into two and time is split into three (eg hr, min, sec)
the one counting the smallest intreval (TDC) is measuring a nm roughly
next counter - BX counter, every 30 TDC count
third counter resets every X of BX counts
with a label of which channel provided the signal 
gives a timing

In [6]:
# 1\. Create a Pandas DataFrame reading N rows of the 
#'data_000637.txt' dataset. Choose N to be smaller than or 
# equal to the maximum number of rows and larger that 10k.

In [1]:
import pandas as pd
import numpy as np

In [26]:
entries = 'data_000637.txt'
#N=30000
df = pd.read_csv(entries)#, nrows=N)
print(df)


         HEAD  FPGA  TDC_CHANNEL   ORBIT_CNT  BX_COUNTER  TDC_MEAS
0           1     0          123  3869200167        2374        26
1           1     0          124  3869200167        2374        27
2           1     0           63  3869200167        2553        28
3           1     0           64  3869200167        2558        19
4           1     0           64  3869200167        2760        25
...       ...   ...          ...         ...         ...       ...
1310715     1     0           62  3869211171         762        14
1310716     1     1            4  3869211171         763        11
1310717     1     0           64  3869211171         764         0
1310718     1     0          139  3869211171         769         0
1310719     1     0           61  3869211171         762        18

[1310720 rows x 6 columns]


In [3]:
# 2\. Find out the number of BX in a ORBIT (the value 'x').
#number of BX for ORBIT to increase by 1

In [21]:
print(df[42:130])
#max BX is 3187 in row 42 and 3538 in row 127
#127-42 = 85
x = df["BX_COUNTER"].max()+1
print(x)

     HEAD  FPGA  TDC_CHANNEL   ORBIT_CNT  BX_COUNTER  TDC_MEAS       time_ns  \
42      1     0           57  3869200167        3187        17  79689.166667   
43      1     1            1  3869200168           7        18    190.000000   
44      1     0           64  3869200168           7        16    188.333333   
45      1     1            1  3869200168          13        18    340.000000   
46      1     0           63  3869200168           8        28    223.333333   
..    ...   ...          ...         ...         ...       ...           ...   
125     1     0           59  3869200168        1885        14  47136.666667   
126     1     0          139  3869200168        1894         0  47350.000000   
127     1     0           62  3869200168        3538         4  88453.333333   
128     1     0           74  3869200169           1        18     40.000000   
129     1     1           16  3869200169         131        11   3284.166667   

     HEAD_r  
42        0  
43        0

In [22]:
#3\. Find out how much the data taking lasted. 
#You can either make an estimate based on the fraction 
#of the measurements (rows) you read, or perform this check 
#precisely by reading out the whole dataset.

In [23]:
#no rows in df = 1310720
#amount of time taken = (ORB_CNT final row - ORB_CNT first row)*3564*25 in ns
print(df.head(1)["ORBIT_CNT"])
print(df.tail(1)["ORBIT_CNT"])
t_orb = df.iloc[1310719,3]-df.iloc[0,3]
t_bx = t_orb * 3564
t_ns = t_bx * 25

print(t_ns, "ns")
t_s = t_ns / 10 ** 9
print(t_s, "seconds")

0    3869200167
Name: ORBIT_CNT, dtype: int64
1310719    3869211171
Name: ORBIT_CNT, dtype: int64
980456400 ns
0.9804564 seconds


In [24]:
#4\. Create a new column with the absolute time in ns 
#(as a combination of the other three columns with timing information).

In [36]:
#absolute time - ORB_CNT:BX_COUNTER:TDC_MEAS 
#hr:m:s

df['time_ns']=df['TDC_MEAS']*25/30+df['BX_COUNTER']*25
print(df.head())


   HEAD  FPGA  TDC_CHANNEL   ORBIT_CNT  BX_COUNTER  TDC_MEAS       time_ns
0     1     0          123  3869200167        2374        26  59371.666667
1     1     0          124  3869200167        2374        27  59372.500000
2     1     0           63  3869200167        2553        28  63848.333333
3     1     0           64  3869200167        2558        19  63965.833333
4     1     0           64  3869200167        2760        25  69020.833333


In [9]:
#5\. Replace the values (all 1) of the HEAD column randomly with 0 or 1.

In [32]:
HEAD_r = np.random.randint(0,2,1310720)
HEAD_r = HEAD_r.reshape(1310720, 1)

df_mod = df.copy()
df_mod['HEAD_r'] = HEAD_r
#print(df_mod)
df_mod.drop(columns = ['HEAD'])
df_mod = df_mod[['HEAD_r', 'FPGA', 'TDC_CHANNEL', 'ORBIT_CNT', 'BX_COUNTER', 'TDC_MEAS', 'time_ns']]
print(df_mod.head())
print(df.head())

   HEAD_r  FPGA  TDC_CHANNEL   ORBIT_CNT  BX_COUNTER  TDC_MEAS       time_ns
0       0     0          123  3869200167        2374        26  59371.666667
1       0     0          124  3869200167        2374        27  59372.500000
2       1     0           63  3869200167        2553        28  63848.333333
3       0     0           64  3869200167        2558        19  63965.833333
4       1     0           64  3869200167        2760        25  69020.833333
   HEAD  FPGA  TDC_CHANNEL   ORBIT_CNT  BX_COUNTER  TDC_MEAS       time_ns
0     1     0          123  3869200167        2374        26  59371.666667
1     1     0          124  3869200167        2374        27  59372.500000
2     1     0           63  3869200167        2553        28  63848.333333
3     1     0           64  3869200167        2558        19  63965.833333
4     1     0           64  3869200167        2760        25  69020.833333


In [11]:
#6\. Create a new DataFrame that contains only the rows with HEAD=1.

In [33]:
df_new = df_mod[df_mod['HEAD_r'] > 0]
#df_new.dropna(how = 'any')
print(df_new.head())

    HEAD_r  FPGA  TDC_CHANNEL   ORBIT_CNT  BX_COUNTER  TDC_MEAS       time_ns
2        1     0           63  3869200167        2553        28  63848.333333
4        1     0           64  3869200167        2760        25  69020.833333
10       1     1            7  3869200167        2785         4  69628.333333
11       1     0           64  3869200167        2786        19  69665.833333
14       1     0           56  3869200167        2789         3  69727.500000


In [13]:
#7\. Make two occupancy plots (one for each FPGA), 
#i.e. plot the number of counts per TDC channel